### Part 1

In [ ]:
## Install the required libraries
%pip install requests
%pip install openpyxl

In [ ]:
## Loading libraries
import json
from urllib.request import urlopen
import csv
import pandas as pd

In [ ]:
## Loading the json file and viewing the contents of the data
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"

with urlopen(url) as response:
    source = response.read()
    
data = json.loads(source)
print(json.dumps(data, indent=2))


In [ ]:
## Understanding the structure of the data better
#len(data)
#len(data[0])
#len(data[0]["restaurants"])

In [ ]:
## Getting the Countries mapped to each Country Code
df = pd.read_excel('Country-Code.xlsx')
country_codes = dict(zip(df['Country Code'], df['Country'])) 

In [ ]:
## Extracing the relevant fields
restaurants = []

for i in data:
    for j in i["restaurants"]:
        country_id = j["restaurant"]["location"]["country_id"]
        if country_id in country_codes:
            country = country_codes[country_id]
        else:
            country = "NA"
        entry = [j["restaurant"]["id"], j["restaurant"]["name"], country, 
                 j["restaurant"]["location"]["city"], j["restaurant"]["user_rating"]["votes"], 
                 float(j["restaurant"]["user_rating"]["aggregate_rating"]), j["restaurant"]["cuisines"]]
        restaurants.append(entry)
        
    

In [ ]:
## Coverting to csv file
csvheader = ["RESTAURANT ID", "RESTAURANT NAME", "COUNTRY", "CITY",
             "USER RATING VOTES", "USER AGGREGATE RATING", "CUISINES"]

with open("restaurants.csv", "w", encoding="UTF-8", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(csvheader)
    writer.writerows(restaurants)

### Part 2

In [ ]:
## Install libraries
%pip install pandas

In [ ]:
## Loading libraries to be used
import pandas as pd
from datetime import datetime

In [ ]:
## Extracting the relevant fields for restaurants that have past events in the month of April 2019
events = []

april = pd.date_range(start='4/1/2019', end='4-30-2019').date
format = '%Y-%m-%d'

for i in data:
    for j in i["restaurants"]:

        if "zomato_events" in j["restaurant"]:
            for k in j["restaurant"]["zomato_events"]:
                start = datetime.strptime(k["event"]["start_date"], format).date()
                end = datetime.strptime(k["event"]["end_date"], format).date()
                
                if (start in april or end in april) or (start < april[0] and end > april[-1]):
                    url = ', \n'.join([m["photo"]["url"] for m in k["event"]["photos"]])
                    entry = [k["event"]["event_id"], j["restaurant"]["id"], j["restaurant"]["name"], 
                             url, k["event"]["title"], start, end]
                    events.append(entry)

In [ ]:
## Coverting to csv file
import csv
csvheader = ["EVENT ID", "RESTAURANT ID", "RESTAURANT NAME", "PHOTO URL",
             "EVENT TITLE", "EVENT START DATE", "EVENT END DATE"]

with open("restaurants_events.csv", "w", encoding="UTF-8", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(csvheader)

    # populating empty cells with N.A.
    for event in events:
        event = [e if e != "" else 'N.A.' for e in event]
        writer.writerow(event)

### Part 3

In [40]:
## Getting the thresholds for different text ratings
text = ["Excellent", "Very Good", "Good", "Average", "Poor"]
threshold = {"Excellent": 5.0, "Very Good": 5.0,"Good": 5.0, "Average": 5.0, "Poor": 5.0}

for i in data:
    for j in i["restaurants"]:
        rating_text = j["restaurant"]["user_rating"]["rating_text"]
        aggregate_rating = float(j["restaurant"]["user_rating"]["aggregate_rating"])
        if rating_text in text:
            if rating_text in threshold:
                if aggregate_rating < threshold[rating_text]:
                    threshold[rating_text] = aggregate_rating

print(threshold)

{'Excellent': 4.5, 'Very Good': 4.0, 'Good': 3.5, 'Average': 2.5, 'Poor': 2.2}


However, assuming that the lowest rating is Poor, the threshold should be 0 instead. Thus, threshold of different rating text based on aggregate rating: 

Excellent: 4.5 ≤ x ≤ 5.0

Very Good: 4.0 ≤ x < 4.5

Good: 3.5 ≤ x < 4.0

Average: 2.5 ≤ x  < 3.5

Poor: x < 2.5